In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np

### 处理丢失数据
- 有两种丢失数据：
    - None
    - np.nan(NaN)

- 两种丢失数据的区别

In [2]:
type(None)

NoneType

In [3]:
type(np.nan)

float

- 为什么在数据分析中需要用到的是浮点类型的空而不是对象类型？
    - 数据分析中会常常使用某些形式的运算来处理原始数据，如果原数数据中的空值为NAN的形式，则不会干扰或者中断运算。
    - NAN可以参与运算的
    - None是不可以参与运算

In [4]:
np.nan + 1

nan

In [5]:
None + 1

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

- 在pandas中如果遇到了None形式的空值则pandas会将其强转成NAN的形式。

In [7]:
df = DataFrame(data=np.random.randint(0,100,size=(7,5)))
df.iloc[2,3] = None
df.iloc[4,2] = np.nan
df.iloc[5,4] = None
df

,0,1,2,3,4
0,23,67,72.0,72.0,78.0
1,37,26,28.0,44.0,19.0
2,0,69,90.0,NaN,28.0
3,76,67,15.0,15.0,50.0
4,54,52,NaN,69.0,53.0
5,38,85,0.0,80.0,NaN
6,86,98,88.0,70.0,84.0


### pandas处理空值操作
- isnull
- notnull
- any
- all
- dropna
- fillna

- 方式1：对空值进行过滤（删除空所在的行数据）
    - 技术：isnull，notnull，any，all

In [10]:
df.isnull()

,0,1,2,3,4
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,True,False
3,False,False,False,False,False
4,False,False,True,False,False
5,False,False,False,False,True
6,False,False,False,False,False


In [9]:
#哪些行中有空值
#any(axis=1)检测哪些行中存有空值
df.isnull().any(axis=1) #any会作用isnull返回结果的每一行
#true对应的行就是存有缺失数据的行

0    False
1    False
2     True
3    False
4     True
5     True
6    False
dtype: bool

In [11]:
df.notnull()

,0,1,2,3,4
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,False,True
3,True,True,True,True,True
4,True,True,False,True,True
5,True,True,True,True,False
6,True,True,True,True,True


In [12]:
df.notnull().all(axis=1)

0     True
1     True
2    False
3     True
4    False
5    False
6     True
dtype: bool

In [13]:
#将布尔值作为源数据的行索引
df.loc[df.notnull().all(axis=1)]

,0,1,2,3,4
0,23,67,72.0,72.0,78.0
1,37,26,28.0,44.0,19.0
3,76,67,15.0,15.0,50.0
6,86,98,88.0,70.0,84.0


In [18]:
#获取空对应的行数据
df.loc[df.isnull().any(axis=1)]
#获取空对应行数据的行索引
indexs = df.loc[df.isnull().any(axis=1)].index
indexs

Int64Index([2, 4, 5], dtype='int64')

In [19]:
df.drop(labels=indexs,axis=0)

,0,1,2,3,4
0,23,67,72.0,72.0,78.0
1,37,26,28.0,44.0,19.0
3,76,67,15.0,15.0,50.0
6,86,98,88.0,70.0,84.0


- 方式2：
    - dropna：可以直接将缺失的行或者列进行删除

In [23]:
df.dropna(axis=0)

,0,1,2,3,4
0,23,67,72.0,72.0,78.0
1,37,26,28.0,44.0,19.0
3,76,67,15.0,15.0,50.0
6,86,98,88.0,70.0,84.0


- 对缺失值进行覆盖
    - fillna

In [25]:
df.fillna(value=999) #使用指定值将源数据中所有的空值进行填充

,0,1,2,3,4
0,23,67,72.0,72.0,78.0
1,37,26,28.0,44.0,19.0
2,0,69,90.0,999.0,28.0
3,76,67,15.0,15.0,50.0
4,54,52,999.0,69.0,53.0
5,38,85,0.0,80.0,999.0
6,86,98,88.0,70.0,84.0


In [28]:
#使用空的近邻值进行填充
#method=ffill向前填充，bfill向后填充
df.fillna(axis=0,method='bfill')

,0,1,2,3,4
0,23,67,72.0,72.0,78.0
1,37,26,28.0,44.0,19.0
2,0,69,90.0,15.0,28.0
3,76,67,15.0,15.0,50.0
4,54,52,0.0,69.0,53.0
5,38,85,0.0,80.0,84.0
6,86,98,88.0,70.0,84.0


- 什么时候用dropna什么时候用fillna
    - 尽量使用dropna，如果删除成本比较高，则使用fillna

- 使用空值对应列的均值进行空值填充

In [36]:
for col in df.columns:
    #检测哪些列中存有空值
    if df[col].isnull().sum() > 0:#说明df[col]中存有空值
        mean_value = df[col].mean()
        df[col] = df[col].fillna(value=mean_value)

### 面试题
- 数据说明： 
    - 数据是1个冷库的温度数据，1-7对应7个温度采集设备，1分钟采集一次。

- 数据处理目标：
    - 用1-4对应的4个必须设备，通过建立冷库的温度场关系模型，预估出5-7对应的数据。
    - 最后每个冷库中仅需放置4个设备，取代放置7个设备。
    - f(1-4) --> y(5-7)

- 数据处理过程：
    - 1、原始数据中有丢帧现象，需要做预处理；
    - 2、matplotlib 绘图；
    - 3、建立逻辑回归模型。

- 无标准答案，按个人理解操作即可，请把自己的操作过程以文字形式简单描述一下，谢谢配合。

- 测试数据为testData.xlsx


In [40]:
data = pd.read_excel('./data/testData.xlsx').drop(labels=['none','none1'],axis=1)
data

,time,1,2,3,4,5,6,7
0,2019-01-27 17:00:00,-24.8,-18.2,-20.8,-18.8,NaN,NaN,NaN
1,2019-01-27 17:01:00,-23.5,-18.8,-20.5,-19.8,-15.2,-14.5,-16.0
2,2019-01-27 17:02:00,-23.2,-19.2,NaN,NaN,-13.0,NaN,-14.0
3,2019-01-27 17:03:00,-22.8,-19.2,-20.0,-20.5,NaN,-12.2,-9.8
4,2019-01-27 17:04:00,-23.2,-18.5,-20.0,-18.8,-10.2,-10.8,-8.8
5,2019-01-27 17:05:00,NaN,NaN,-19.0,-18.2,-10.0,-10.5,-10.8
6,2019-01-27 17:06:00,NaN,-18.5,-18.2,-17.5,NaN,NaN,NaN
7,2019-01-27 17:07:00,-24.8,-18.0,-17.5,-17.2,-14.2,-14.0,-12.5
8,2019-01-27 17:08:00,-25.2,-17.8,NaN,NaN,-16.2,NaN,-14.5
9,2019-01-27 17:09:00,-24.8,-18.2,NaN,-17.5,NaN,-15.5,-16.0


In [41]:
data.shape

(1060, 8)

In [43]:
#删除空对应的行数据
data.dropna(axis=0).shape

(927, 8)

In [45]:
#填充
data.fillna(method='ffill',axis=0).fillna(method='bfill',axis=0)

,time,1,2,3,4,5,6,7
0,2019-01-27 17:00:00,-24.8,-18.2,-20.8,-18.8,-15.2,-14.5,-16.0
1,2019-01-27 17:01:00,-23.5,-18.8,-20.5,-19.8,-15.2,-14.5,-16.0
2,2019-01-27 17:02:00,-23.2,-19.2,-20.5,-19.8,-13.0,-14.5,-14.0
3,2019-01-27 17:03:00,-22.8,-19.2,-20.0,-20.5,-13.0,-12.2,-9.8
4,2019-01-27 17:04:00,-23.2,-18.5,-20.0,-18.8,-10.2,-10.8,-8.8
5,2019-01-27 17:05:00,-23.2,-18.5,-19.0,-18.2,-10.0,-10.5,-10.8
6,2019-01-27 17:06:00,-23.2,-18.5,-18.2,-17.5,-10.0,-10.5,-10.8
7,2019-01-27 17:07:00,-24.8,-18.0,-17.5,-17.2,-14.2,-14.0,-12.5
8,2019-01-27 17:08:00,-25.2,-17.8,-17.5,-17.2,-16.2,-14.0,-14.5
9,2019-01-27 17:09:00,-24.8,-18.2,-17.5,-17.5,-16.2,-15.5,-16.0


### 处理重复数据

In [48]:
df = DataFrame(data=np.random.randint(0,100,size=(8,6)))
df.iloc[1] = [1,1,1,1,1,1]
df.iloc[3] = [1,1,1,1,1,1]
df.iloc[5] = [1,1,1,1,1,1]
df

,0,1,2,3,4,5
0,3,29,47,11,69,7
1,1,1,1,1,1,1
2,6,43,65,79,52,82
3,1,1,1,1,1,1
4,2,67,90,8,96,76
5,1,1,1,1,1,1
6,38,6,56,50,71,30
7,9,16,12,67,32,0


In [49]:
#检测哪些行存有重复的数据
df.duplicated(keep='first')

0    False
1    False
2    False
3     True
4    False
5     True
6    False
7    False
dtype: bool

In [51]:
df.loc[~df.duplicated(keep='first')]

,0,1,2,3,4,5
0,3,29,47,11,69,7
1,1,1,1,1,1,1
2,6,43,65,79,52,82
4,2,67,90,8,96,76
6,38,6,56,50,71,30
7,9,16,12,67,32,0


In [56]:
#异步到位删除
df.drop_duplicates(keep='first')

,0,1,2,3,4,5
0,3,29,47,11,69,7
1,1,1,1,1,1,1
2,6,43,65,79,52,82
4,2,67,90,8,96,76
6,38,6,56,50,71,30
7,9,16,12,67,32,0


### 处理异常数据
- 自定义一个1000行3列（A，B，C）取值范围为0-1的数据源，然后将C列中的值大于其两倍标准差的异常值进行清洗

In [58]:
df = DataFrame(data=np.random.random(size=(1000,3)),columns=['A','B','C'])
df.head()

,A,B,C
0,0.093731,0.358117,0.599607
1,0.415552,0.124499,0.820207
2,0.278742,0.222851,0.786832
3,0.537346,0.339687,0.276611
4,0.414794,0.179321,0.094958


In [60]:
#制定判定异常值的条件
twice_std = df['C'].std() * 2
twice_std

0.5664309886908782

In [64]:
df.loc[~(df['C'] > twice_std)]

,A,B,C
3,0.537346,0.339687,0.276611
4,0.414794,0.179321,0.094958
5,0.397169,0.610316,0.420824
7,0.740718,0.730160,0.302804
8,0.483627,0.146781,0.126308
9,0.530481,0.426309,0.126464
11,0.450225,0.399873,0.564089
12,0.149199,0.706758,0.271892
14,0.503967,0.098280,0.239464
15,0.630785,0.661415,0.139548
